## Linear Regression: Model Selection and Cross-Validation

Functions

`RandomState.permute`, `sm.OLS`, `set`, `scipy.random.norm.ppf`, `np.linspace`, `np.mean`

### Exercise 39
Four portfolios we have been looking at, and considering all 8 sets of
regressors which range from no factor to all 3 factors, which model is preferred
by AIC, BIC, GtS and StG?

In [7]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
from itertools import product

ff = pd.read_hdf("data/ff.h5","ff")
factors = sm.add_constant(ff.iloc[:,:3])
portfolios = ff.iloc[:,4:]
portfolios = portfolios[["SMALL LoBM","SMALL HiBM","BIG LoBM", "BIG HiBM"]]

In [9]:
true_false = [True, False]
params = [true_false] * 3
choices = list(product(*params))
choices

[(True, True, True),
 (True, True, False),
 (True, False, True),
 (True, False, False),
 (False, True, True),
 (False, True, False),
 (False, False, True),
 (False, False, False)]

In [11]:
# AIC and BIC selection

for column in portfolios:
    results = {}
    for i in range(8):
        sel = [True] + list(choices[i])
        x = factors.loc[:,sel]
        regressor_names = []
        for j in range(4):
            if sel[j]:
                regressor_names.append(factors.columns[j])
        names = tuple(regressor_names)
        res = sm.OLS(portfolios[column],x).fit()
        results[names] = [res.aic, res.bic] 
    ic = pd.DataFrame(results, index = ["AIC", "BIC"]).T

    aic_model = ic.sort_values("AIC").index[0]
    bic_model = ic.sort_values("BIC").index[0]
    print(f"For {column}, AIC selects {aic_model}")
    print(f"For {column}, BIC selects {bic_model}")


For SMALL LoBM, AIC selects ('const', 'Mkt-RF', 'SMB', 'HML')
For SMALL LoBM, BIC selects ('const', 'Mkt-RF', 'SMB', 'HML')
For SMALL HiBM, AIC selects ('const', 'Mkt-RF', 'SMB', 'HML')
For SMALL HiBM, BIC selects ('const', 'Mkt-RF', 'SMB', 'HML')
For BIG LoBM, AIC selects ('const', 'Mkt-RF', 'SMB', 'HML')
For BIG LoBM, BIC selects ('const', 'Mkt-RF', 'SMB', 'HML')
For BIG HiBM, AIC selects ('const', 'Mkt-RF', 'SMB', 'HML')
For BIG HiBM, BIC selects ('const', 'Mkt-RF', 'SMB', 'HML')


In [13]:
# General to Specific selection
from scipy import stats
cv = stats.norm.ppf(.995)

for column in portfolios:
    included  = list(factors.columns)
    y = portfolios[column]
    for i in range(3):
        x = factors.loc[:,included]
        res = sm.OLS(y,x).fit(cov_type="HC0")
        tstats = res.tvalues
        if tstats.abs().min() < cv:
            sorted_tstats = tstats.abs().sort_values()
            remove = sorted_tstats.index[0]
            included.remove(remove)
        else:
            break
    print(f"For {column}, GtS selects {included}")


For SMALL LoBM, GtS selects ['Mkt-RF', 'SMB']
For SMALL HiBM, GtS selects ['const', 'Mkt-RF', 'SMB', 'HML']
For BIG LoBM, GtS selects ['const', 'Mkt-RF', 'SMB', 'HML']
For BIG HiBM, GtS selects ['Mkt-RF', 'HML']


In [15]:
# Specific to general approach

for column in portfolios:
    included = []
    excluded = factors.columns
    y = portfolios[column]
    for i in range(3):
        tstats = {}
        for next_var in excluded:
            col_names = included + [next_var]
            x = factors.loc[:,col_names]
            res = sm.OLS(y,x).fit(cov_type="HC0")
            tstats[next_var] = res.tvalues.iloc[-1]
        tstats = pd.Series(tstats)
        if tstats.abs().max() > cv:
            sorted_tstats = tstats.abs().sort_values()
            included = included + [sorted_tstats.index[-1]]
        else:
            break
        excluded = set(factors.columns).difference(included)
    print(f"For {column}, StG selects {included}, leaving out {excluded}")

For SMALL LoBM, StG selects ['SMB', 'Mkt-RF'], leaving out {'const', 'HML'}
For SMALL HiBM, StG selects ['Mkt-RF', 'SMB', 'HML'], leaving out {'const'}
For BIG LoBM, StG selects ['Mkt-RF', 'HML', 'const'], leaving out {'SMB'}
For BIG HiBM, StG selects ['Mkt-RF', 'HML'], leaving out {'const', 'SMB'}


### Exercise 40
Cross-validation is a method of analyzing the in-sample forecasting ability of a
cross-sectional model by using $\alpha\%$ of the data to estimate the model and
then measuring the fit using the remaining $100-\alpha\%$. The most common forms
are 5- and 10-fold cross-validation which use $\alpha=20\%$ and $10\%$, respectively.
k-fold cross validation is implemented by randomly grouping the data into
k-equally-sized groups, using k-1 of the groups to estimate parameters, and
then evaluating using the bin that was held out. This is then repeated so that
each bin is held out.

1. Implement cross-validation using the 5- and 10-fold methods for all 8 models.
2. For each model, compute the full-sample sum of squared errors as well as the
   sum-of-squared errors using the held-out sample. Note that all data points
   will appear exactly once in both of these sum or squared errors. What happens
   to the cross-validated $R^{2}$ when computed on the held out sample when compared
   to the full-sample $R^{2}$? (k-fold cross validated SSE by the TSS).

In [60]:
rs = np.random.RandomState(1)
folds = 10
nobs = portfolios.shape[0]
order = list(rs.permutation(nobs))
block = nobs / folds

for column in portfolios:
    portfolio = portfolios[column]
    model_errors = {}
    for j in range(8):
        sel = [True] + list(choices[j])
        model_factors = factors.loc[: , sel]
        errors = portfolio.copy()
        for i in range(folds):
            include = order[:int(i*block)] + order[int((i+1)*block):]
            hold_out = order[int(i*block):int((i+1)*block)]
            y = portfolio.iloc[include]
            x = model_factors.iloc[include]
            mod = sm.OLS(y,x)
            res = mod.fit(cov_type="HC0")
            y_hat = res.predict(model_factors.iloc[hold_out])
            err = portfolio.iloc[hold_out] - y_hat
            errors.loc[err.index] = err
        model_name = tuple(factors.columns[sel])
        model_errors[model_name] = errors
    model_errors = pd.DataFrame(model_errors)
    mse = (model_errors ** 2).mean()
    mse = mse.sort_values()

    # get rid of missing values
    selected_factors = pd.Series(mse.index[0]).dropna()
    x = factors[selected_factors]
    res = sm.OLS(portfolios[column], x).fit(cov_type="HC0")
    print()
    print(f"For {column}, CV selects {tuple(selected_factors)}")
    print(f"The MSEs are {mse.iloc[0]} (CV) and {res.mse_resid} (full sample)")
    print(f"The R2s are {1 - mse.iloc[0] / res.mse_total} (CV) and {1 - res.mse_resid / res.mse_total} (full sample)")



For SMALL LoBM, CV selects ('const', 'Mkt-RF', 'SMB', 'HML')
The MSEs are 41.73800385617749 (CV) and 39.84597408114153 (full sample)
The R2s are 0.7045162799136672 (CV) and 0.7179108830280897 (full sample)

For SMALL HiBM, CV selects ('const', 'Mkt-RF', 'SMB', 'HML')
The MSEs are 5.462914480426759 (CV) and 5.223322832919042 (full sample)
The R2s are 0.9356165672517653 (CV) and 0.9384402857594633 (full sample)

For BIG LoBM, CV selects ('const', 'Mkt-RF', 'SMB', 'HML')
The MSEs are 1.3875348945923307 (CV) and 1.3694629243223586 (full sample)
The R2s are 0.9509545133244749 (CV) and 0.9515933070445686 (full sample)

For BIG HiBM, CV selects ('const', 'Mkt-RF', 'SMB', 'HML')
The MSEs are 12.049970805233622 (CV) and 11.686213642228015 (full sample)
The R2s are 0.8329301822673112 (CV) and 0.8379735839406091 (full sample)
